# Tutorial - Axial pile capacity calculation according to API RP2 GEO with groundhog

At the request of users, a routine for axial capacity is developed within groundhog.

The pile capacity class ``AxCapCalculation`` develops a workflow in which the unit skin friction methods and unit end bearing methods are introduced into a ``SoilProfile`` object.

A check is then performed to assess whether the necessary soil parameters are defined in the ``SoilProfile``. If all necessary parameters are available, a calculation grid is defined and the unit skin friction and unit end bearing calculations are calculated. This notebook outlines the different components of the calculation for the API RP2 GEO Main Text method.

## Library imports



In [ ]:
import numpy as np

In [ ]:
import plotly.io as pio
pio.templates.default = 'plotly_white'

In [ ]:
from groundhog.general.soilprofile import SoilProfile
from groundhog.general.plotting import LogPlot

## Preparing a ``groundhog`` ``SoilProfile`` object for axial capacity analysis

### Definition of methods and soil parameters in ``SoilProfile``

For each layer in the ``SoilProfile``, the unit skin friction and unit end bearing need to be set. Each methods has associated required parameters. The presence of these parameters can be checked before starting the calculation.

First, a basic soil profile with three layers is defined:

In [ ]:
sp = SoilProfile({
    'Depth from [m]': [0, 5, 10],
    'Depth to [m]': [5, 10, 20],
    'Soil type': ['SAND', 'CLAY', 'SAND'],
    'Total unit weight [kN/m3]': [20, 18, 20]
})
sp

Next, the calculation methods for unit skin friction and unit end bearing are specified in the columns ``Unit skin friction`` and ``Unit end bearing``.

In [ ]:
sp['Unit skin friction'] = ['API RP2 GEO Sand', 'API RP2 GEO Clay', 'API RP2 GEO Sand']
sp['Unit end bearing'] = ['API RP2 GEO Sand', 'API RP2 GEO Clay', 'API RP2 GEO Sand']
sp

### Overburden calculation

As several method require the vertical effective stress as an input, the stresses can be calculated using the ``calculate_overburden`` method.

In [ ]:
sp.calculate_overburden()

The vertical effective stress profile can be plotted:

In [ ]:
stress_plot = LogPlot(sp, no_panels=1, fillcolordict={'SAND': 'yellow', 'CLAY': 'brown'})
stress_plot.add_trace(
    x=sp.soilparameter_series('Vertical effective stress [kPa]')[1],
    z=sp.soilparameter_series('Vertical effective stress [kPa]')[0],
    name=r'$ \sigma_{v0}^{\prime} $', showlegend=False, panel_no=1)
stress_plot.set_xaxis(title=r'$ \sigma_{v0}^{\prime} \ \text{[kPa]} $', panel_no=1)
stress_plot.set_zaxis(title=r'$ z \ \text{[m]} $', range=(20, 0))
stress_plot.show()

## Axial pile capacity calculation setup

### Creating the calculation object

An axial capacity calculation can be set up based on the soil profile. The ``AxcapCalculation`` class contains the necessary functionality.

In [ ]:
from groundhog.deepfoundations.axialcapacity.axcap import AxCapCalculation

In [ ]:
calc = AxCapCalculation(sp)

### Checking of presence of require soil parameters

The presence of the required input to unit skin friction and unit end bearing calculations can be checked. Note that both numerical and string soil parameters need to be checked:

In [ ]:
from groundhog.general.parameter_mapping import SOIL_PARAMETER_MAPPING, reverse_dict
from groundhog.deepfoundations.axialcapacity.skinfriction import SKINFRICTION_PARAMETERS, SKINFRICTION_METHODS
from groundhog.deepfoundations.axialcapacity.endbearing import ENDBEARING_PARAMETERS, ENDBEARING_METHODS

``groundhog`` has a standard mapping for soil parameters, which defines the mapping from column headers in a Pandas dataframe to ``groundhog`` function variables. Column headers must conform to this naming.

In [ ]:
SOIL_PARAMETER_MAPPING

The necessary parameters for skin friction calculation can be printed. In v0.9.0, only the API RP2 GEO methods are defined with their associated parameters.

In [ ]:
SKINFRICTION_PARAMETERS

With the method ``check_methods``, the presence of the parameters can be checked. At this point, the parameters are not yet defined so the method will return a message that the required parameters are not found.

In [ ]:
calc.check_methods()

The required soil parameters can be defined. Example parameters are defined below. The documentation of the unit skin friction and unit end bearing methods can be checked in the ``groundhog`` documentation.

In [ ]:
calc.sp['API soil description'] = ['Sand-silt', None, 'Sand']
calc.sp['API relative density description'] = ['Medium dense', None, 'Dense']
calc.sp['Undrained shear strength from [kPa]'] = [np.nan, 100, np.nan]
calc.sp['Undrained shear strength to [kPa]'] = [np.nan, 150, np.nan]

When re-running the ``check_methods`` no messages are returned. But the attribute ``checked`` is now ``True``.

In [ ]:
calc.check_methods()
calc.checked

## Gridding for axial capacity calculation

A calculation grid can be created based on the soil profile with a spacing of 1m.

In [ ]:
calc.create_grid(dz=1)

The grid has a ``nodes`` and ``elements`` attribute where the former contains the nodal coordinates and the associated parameter values at the nodes. The latter contains the element definition with the parameter values at the center of the element.

In [ ]:
calc.grid.nodes.head()

In [ ]:
calc.grid.elements.head()

The parameters in the grid can be prepared for plotting by applying the ``soilparameter_series`` with the name of the parameter under consideration. The x- and z-values can be defined based on the parameter name.

In [ ]:
su_z, su_x = calc.grid.soilparameter_series('Undrained shear strength [kPa]')
sigmav_z, sigmav_x = calc.grid.soilparameter_series('Vertical effective stress [kPa]')

In [ ]:
parameter_plot = LogPlot(calc.sp, no_panels=2, fillcolordict={'SAND': 'yellow', 'CLAY': 'brown'})

parameter_plot.add_trace(x=su_x, z=su_z, showlegend=False, mode='lines',name='Su', panel_no=1)
parameter_plot.add_trace(x=sigmav_x, z=sigmav_z, showlegend=True, mode='lines',name='With linear', panel_no=2)

parameter_plot.set_xaxis(title=r'$ \sigma_{vo}^{\prime} \ \text{[kPa]} $', panel_no=2, range=(0, sigmav_x.max()))
parameter_plot.set_xaxis(title=r'$ S_u \ \text{[kPa]} $', panel_no=1, range=(0, 400))
parameter_plot.set_zaxis(title=r'$ z \ \text{[m]}$')
parameter_plot.show()

## Calculation of unit skin friction and unit end bearing

Unit skin friction and unit end bearing can be calculated by running the functions for the selected methods at each element.

Note that the unit skin friction distribution may be depth-dependent, so at this stage, the pile penetration will need to be introduced.

### Setting the pile penetration

The pile penetration can be set with the ``set_pilepenetration`` method. In this example, a pile penetration of 18m is selected.

In [ ]:
calc.set_pilepenetration(18)

### Calculation of unit skin friction

For the calculation of unit skin friction, the equation for the selected method is applied for each element.

In [ ]:
calc.calculate_unitskinfriction()

### Calculation of unit end bearing

Similarly, the unit end bearing can be calculated:

In [ ]:
calc.calculate_unitendbearing()

### Unit skin friction and unit end bearing visualisation

The unit skin friction and unit end bearing profile can be visualised with the ``groundhog`` ``LogPlot``.

In [ ]:
z_fs, x_fs = calc.output.soilparameter_series('Unit skin friction outside compression [kPa]')
z_qb, x_qb = calc.output.soilparameter_series('Unit end bearing plugged [kPa]')

fs_qb_plot = LogPlot(calc.output, no_panels=2, fillcolordict={'SAND': 'yellow', 'CLAY': 'brown'})

fs_qb_plot.add_trace(x=x_fs, z=z_fs, showlegend=False, mode='lines',name='fs comp', panel_no=1)
fs_qb_plot.add_trace(x=x_qb, z=z_qb, showlegend=False, mode='lines',name='qb', panel_no=2)

fs_qb_plot.set_xaxis(title=r'$ q_b \ \text{[kPa]} $', panel_no=2, range=(0, x_qb.max()))
fs_qb_plot.set_xaxis(title=r'$ f_s \ \text{[kPa]} $', panel_no=1, range=(0, x_fs.max()))
fs_qb_plot.set_zaxis(title=r'$ z \ \text{[m]}$')
fs_qb_plot.show()

## Integration of unit skin friction and unit end bearing

Unit skin friction can be integrated over the annulus of the pile. Four modes are discerned:

   - Plugged compression: Outside skin friction and end bearing over the full diameter of the pile
   - Plugged tension: Outside skin friction in tension + weight of the pile material and/or internal soil plug
   - Coring compression: Outside skin friction + inside skin friction + annular end bearing
   - Coring tension: Outside skin friction in tension + inside skin friction in tension
   
The pile circumference and base cross-sectional area need to be known. Because pile shapes and sizes may differ, the weight of the pile and/or internal soil plug is excluded from the assessment. Calculating this component with a separate calculation is straightforward.

In this example, a tubular open-ended pile with 2.48m outer diameter and 50mm wall thickness is taken.

In [ ]:
calc.calculate_pilecapacity(
    circumference=np.pi * 2.48, base_area=0.25 * np.pi * 2.48 ** 2,
    internal_circumference=np.pi * 2.38, annulus_area=0.25 * np.pi * (2.48 ** 2 - 2.38 ** 2))

In [ ]:
calc.output.tail()

### Results for the final penetration

As unit skin friction profiles can be penetration-dependent, the plot of unit skin friction $ f_s $ is only valid for the selected final pile penetration.

The profiles of unit skin friction $ f_s $, unit end bearing $ q_b $ and integrated shaft resistance in compression and tension over the inside and outside of the pile are plotted as well as the plugged and unplugged (coring) end bearing.

In [ ]:
calc.plot_single_penetration()

Numerical values of the results are available in the ``result`` attribute. These values are valid for the final pile penetration.

In [ ]:
calc.result

### Results for all penetrations

The calculation of axial pile capacity needs to be repeated for every pile penetration for pile sizing as unit skin friction and unit end bearing can be depth-dependent (e.g. due to friction fatigue).

In [ ]:
calc.calculate_capacity_profile(
    circumference=np.pi * 2.48, base_area=0.25 * np.pi * 2.48 ** 2,
    internal_circumference=np.pi * 2.38, annulus_area=0.25 * np.pi * (2.48 ** 2 - 2.38 ** 2),
    pile_weight_permeter=0.25 * np.pi * (2.48 ** 2 - 2.38 ** 2) * 68.5,
    soilplug_weight_permeter=0.25 * np.pi * 2.38 ** 2 * 10)

The results can be visualised. The leftmost panel shows the shaft resistance in compression and tension for plugged (only outside) and coring (inside and outside) conditions. The coring and plugged base resistance are shown in the middle panel. In the right-most panel, the compression and tension resistance for plugged and coring conditions are shown. Depending on the relative magnitude of internal shaft resistance vs end bearing on the plug, the calculation selects whether plugged (internal shaft resistance > end bearing on soil plug) or coring (internal shaft resistance < end bearing on soil plug) conditions are expected.

In [ ]:
calc.plot_all_penetrations()

The results are shown in a dataframe which can be exported to csv or Excel.

In [ ]:
calc.capacity_profile.head()